In [2]:
import cv2
import face_recognition
import datetime
from deepface import DeepFace



def largest_face_location(face_locations: list):
    if len(face_locations) != 0:
        largest_face = max(face_locations, key=lambda face: (face[2] - face[0]) * (face[3] - face[1])) # calculates the area of bounding box and selects max one

        return largest_face
    else:
        return None  # Return None if the list is empty
    
word_descriptions = {
    "Radiant": "Glowing like a happy star.",
    "Unattractive": "Like a scarecrow in a tuxedo.",
    "Exquisite": "As fine as a snowflake's lace.",
    "Repulsive": "Beauty is in the eye of the beholder, but this one's a challenge.",
    "Angelic": "Looks straight out of heaven.",
    "Quirky": "A walking carnival of charm.",
    "Mesmerizing": "Holds you in a trance.",
    "Flawed": "Perfection's distant cousin.",
    "Alluring": "Draws admirers like a magnet.",
    "Odd": "A puzzle for the eyes.",
    "Flawless": "Smooth as polished marble.",
    "Unusual": "Stranger than a cat wearing sunglasses indoors.",
    "Enchanting": "Spells beauty with every move.",
    "Distinctive": "One of a kind, no rewind.",
    "Graceful": "Dances through life with ease.",
    "Homely": "Cozy charm that warms hearts.",
    "Stunning": "A jaw-dropping sight to behold.",
    "Plain": "Faces don't get flatter than this.",
    "Ethereal": "Seems almost otherworldly.",
    "Peculiar": "A delightful twist of the ordinary."
}

location, encoding, landmark, timing, countdown = None, None, None, None, None

video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()

    rgb_frame = frame[:,:,::-1]

    face_locations = face_recognition.api.face_locations(rgb_frame) # For face detection

    if len(face_locations) != 0:

        if location == None:
            location = [largest_face_location(face_locations)]
            encoding = face_recognition.api.face_encodings(rgb_frame, location)[0]
            landmark = face_recognition.api.face_landmarks(rgb_frame, location)[0]
            timing = datetime.datetime.now()
            countdown = 5

        else:
            face_encodings = face_recognition.api.face_encodings(rgb_frame, face_locations)
            comparison = face_recognition.api.compare_faces(face_encodings, encoding)
            if True not in comparison:
                location = [largest_face_location(face_locations)]
                encoding = face_recognition.api.face_encodings(rgb_frame, location)[0]
                landmark = face_recognition.api.face_landmarks(rgb_frame, location)[0]
                timing = datetime.datetime.now()
                countdown = 5
            else:
                index = comparison.index(True)
                location = [face_locations[index]]
                encoding = face_encodings[index]
                landmark = face_recognition.api.face_landmarks(rgb_frame, location)[0]
                current_time = datetime.datetime.now()
                if (current_time - timing).total_seconds() >= 1:
                    timing = current_time

                    if isinstance(countdown, int) and countdown > 1:
                        countdown = countdown - 1
                    else:
                        ######################
                        countdown = 'blah blah'
                        ######################

        (top,right,bottom,left) = location[0]
        cv2.rectangle(frame,(left,top),(right,bottom),(0,255,255),2)
        cv2.putText(frame, str(countdown), (right -130, top -50), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4)
        if isinstance(countdown,str):
            cropped_frame = frame[top:bottom, left:right]
            faces = DeepFace.analyze(cropped_frame, actions=['gender', 'emotion'], enforce_detection=False)
            for face_data in faces:
                dominant_gender = face_data['dominant_gender']
                dominant_emotion = face_data['dominant_emotion']

                # Create text to display on the frame
                text = f"Hey {dominant_emotion} {dominant_gender}"

                # Add the text to the frame
                cv2.putText(frame, text, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


    
    else:
        location, encoding, landmark, timing, countdown = None, None, None, None, None
    
    cv2.imshow('rgb_frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  4.54it/s]
